In [6]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

'.env_file' loaded!
ENV 'PROD' configured!


In [7]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_gei_calculo"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

2025-10-24T21:35:35.292556Z [info     ] Using json file settings.      [root] loc=spark_utils_session.py:301
2025-10-24T21:35:35.293793Z [info     ] Exporting default ENV.         [root] loc=spark_utils_session.py:305
2025-10-24T21:35:35.294177Z [info     ] Exporting custom ENVs.         [root] loc=spark_utils_session.py:338
2025-10-24T21:35:35.294631Z [info     ] Building profile 'efd_t2'.     [root] loc=spark_utils_session.py:221
2025-10-24T21:35:35.294993Z [info     ] Not enough info for building the kerberos client. Ignoring it [root] loc=spark_utils_session.py:284


:: loading settings :: url = jar:file:/opt/cloudera/parcels/SPARK3-3.5.4.3.5.7191000.0-30-1.p0.68499982/lib/spark3/jars/ivy-2.5.2.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/tsevero/.ivy2/cache
The jars for the packages stored in: /home/tsevero/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
com.oracle.database.security#oraclepki added as a dependency
com.oracle.database.security#osdt_core added as a dependency
com.oracle.database.security#osdt_cert added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6e2d8bbc-b495-48e5-8b8e-c644532149a0;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.18.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.2 in central
	found com.oracle.database.security#oraclepki;21.18.0.0 in central
	found com.oracle.databa

In [8]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

Hive Session ID = e04d7a30-1aa0-4e44-a524-718f8f84378a


+------------------+
|namespace         |
+------------------+
|anac              |
|bcadastro         |
|bpe               |
|c115              |
|ccc               |
|ccg               |
|cte               |
|default           |
|destda            |
|detran_share      |
|dime              |
|due               |
|efd               |
|fci               |
|gecob             |
|gescol            |
|gessimples        |
|gplam             |
|information_schema|
|malhas            |
+------------------+
only showing top 20 rows



In [9]:
# Bibliotecas para análise e visualização
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, date
from decimal import Decimal

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['font.size'] = 11

# Acesso ao SparkSession
spark = session.sparkSession

# ===================================================================
# ✅ IMPORTAÇÕES "PADRÃO OURO"
# ===================================================================
from pyspark.sql.functions import (
    col, lit, trim, regexp_replace, coalesce, 
    concat, lpad, floor,
    sum as spark_sum, 
    count as spark_count, 
    avg as spark_avg,
    when as spark_when, 
    desc as spark_desc,
    row_number
)
from pyspark.sql.window import Window
# ===================================================================

print("=" * 80)
print("SISTEMA DE ANÁLISE FISCAL - PROCESSAMENTO NFe/CTe")
print("=" * 80)
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")
print("=" * 80)

SISTEMA DE ANÁLISE FISCAL - PROCESSAMENTO NFe/CTe
Iniciado em: 2025-10-24 18:36:03

Spark UI: None


In [10]:
# ===================================================================
# CÉLULA 3: PARÂMETROS DE ENTRADA
# ===================================================================
# DEFINA AQUI OS PARÂMETROS DA ANÁLISE

# Período de análise (formato YYYYMM)
PERIODO_INICIO = 202001  # Janeiro/2020
PERIODO_FIM = 202509     # Setembro/2025

# Lista de CNPJs a serem analisados (cadastro)
# Se vazio, busca da tabela de cadastro
CNPJS_ANALISE = ['32372396000194', '46909678000192']  # Exemplo: ['12345678000190', '98765432000110']

# Se quiser buscar de uma tabela específica:
TABELA_CADASTRO = "usr_sat_ods.vw_ods_contrib"  # Ajuste conforme sua tabela
CAMPO_CNPJ_CADASTRO = "nu_cnpj"  # Campo que contém o CNPJ

print("=" * 80)
print("PARÂMETROS DA ANÁLISE")
print("=" * 80)
print(f"Período: {PERIODO_INICIO} até {PERIODO_FIM}")
print(f"Tabela Cadastro: {TABELA_CADASTRO}")
print(f"CNPJs específicos: {len(CNPJS_ANALISE) if CNPJS_ANALISE else 'Todos da tabela de cadastro'}")
print("=" * 80)

PARÂMETROS DA ANÁLISE
Período: 202001 até 202509
Tabela Cadastro: usr_sat_ods.vw_ods_contrib
CNPJs específicos: 2


In [11]:
# ===================================================================
# CÉLULA 4: CARREGAR CNPJS DO CADASTRO (Refatorado - API)
# ===================================================================
print("\n" + "=" * 80)
print("CARREGANDO CNPJS DO CADASTRO")
print("=" * 80)

if CNPJS_ANALISE:
    # Usar lista manual
    df_cadastro = spark.createDataFrame(
        [(cnpj,) for cnpj in CNPJS_ANALISE], 
        ["cnpj"]
    )
    print(f"✓ Usando {len(CNPJS_ANALISE)} CNPJs fornecidos manualmente")
else:
    # Buscar da tabela usando API de DataFrame
    df_cadastro = (spark.table(TABELA_CADASTRO)
        .select(
            regexp_replace(
                trim(col(CAMPO_CNPJ_CADASTRO).cast("string")), 
                '[^0-9]', ''
            ).alias("cnpj")
        )
        .where(col("cnpj").isNotNull())
        .distinct()
    )
    
    total_cnpjs = df_cadastro.count()
    print(f"✓ Carregados {total_cnpjs} CNPJs da tabela {TABELA_CADASTRO}")

# Criar view temporária
df_cadastro.createOrReplaceTempView("cadastro_cnpj")

# Materializar os CNPJs em cache (MUITO IMPORTANTE para os joins)
print("\nCacheando tabela 'cadastro_cnpj'...")
spark.sql("CACHE TABLE cadastro_cnpj")
total_cnpjs_cache = spark.table("cadastro_cnpj").count()
print(f"✓ Tabela 'cadastro_cnpj' cacheada com {total_cnpjs_cache} registros.")

# Exibir amostra
print("\nAmostra de CNPJs:")
spark.table("cadastro_cnpj").limit(10).show(truncate=False)


CARREGANDO CNPJS DO CADASTRO
✓ Usando 2 CNPJs fornecidos manualmente

Cacheando tabela 'cadastro_cnpj'...


✓ Tabela 'cadastro_cnpj' cacheada com 2 registros.

Amostra de CNPJs:
+--------------+
|cnpj          |
+--------------+
|32372396000194|
|46909678000192|
+--------------+



In [12]:
# ===================================================================
# CÉLULA 5: CRIAR VIEW DE NFe (OTIMIZADA + SKIP PERÍODO CORROMPIDO)
# ===================================================================
print("\n" + "=" * 80)
print("CRIANDO VIEW DE NFe PROCESSADA (OTIMIZADA)")
print("=" * 80)

# REFRESH da tabela NFe primeiro
print("Fazendo REFRESH da tabela nfe.nfe...")
try:
    spark.sql("REFRESH TABLE nfe.nfe")
    print("✓ REFRESH executado")
except Exception as e:
    print(f"⚠ REFRESH falhou: {e}")

# Materializar os CNPJs em cache
spark.sql("CACHE TABLE cadastro_cnpj")
cnpj_list = [row.cnpj for row in spark.sql("SELECT cnpj FROM cadastro_cnpj").collect()]
print(f"CNPJs no cadastro: {len(cnpj_list)}")

# Criar string para usar no IN clause
cnpj_in_clause = "', '".join(cnpj_list)

# ⚠️ AJUSTE CRÍTICO: Pular períodos corrompidos
# Se 202001 está corrompido, começar de 202002
PERIODO_INICIO_AJUSTADO = 202101  # Começar de Janeiro/2021 ao invés de 2020

print(f"\n⚠️ ATENÇÃO: Pulando período 2020 devido a arquivos corrompidos")
print(f"Período ajustado: {PERIODO_INICIO_AJUSTADO} até {PERIODO_FIM}")

query_nfe_view = f"""
CREATE OR REPLACE TEMPORARY VIEW vw_nfe_processada AS
SELECT 
    -- Identificação
    a.chave AS chave_nfe,
    (a.ano_emissao * 100 + a.mes_emissao) AS periodo_ref,
    CONCAT(LPAD(a.mes_emissao, 2, '0'), '/', a.ano_emissao) AS periodo_ref_formatado,
    a.dhemi_orig AS data_hora_emissao,
    
    -- Emitente (limpeza feita uma vez)
    cnpj_emit AS cnpj_emitente,
    a.procnfe.nfe.infnfe.emit.xnome AS razao_social_emitente,
    a.procnfe.nfe.infnfe.emit.xfant AS nome_fantasia_emitente,
    a.procnfe.nfe.infnfe.emit.ie AS ie_emitente,
    a.procnfe.nfe.infnfe.emit.crt AS crt_emitente,
    a.procnfe.nfe.infnfe.emit.enderemit.uf AS uf_emitente,
    a.procnfe.nfe.infnfe.emit.enderemit.xmun AS municipio_emitente,
    
    -- Destinatário (limpeza feita uma vez)
    cnpj_dest AS cnpj_destinatario,
    a.procnfe.nfe.infnfe.dest.xnome AS razao_social_destinatario,
    a.procnfe.nfe.infnfe.dest.ie AS ie_destinatario,
    a.procnfe.nfe.infnfe.dest.indiedest AS indicador_ie_destinatario,
    a.procnfe.nfe.infnfe.dest.enderdest.uf AS uf_destinatario,
    a.procnfe.nfe.infnfe.dest.enderdest.xmun AS municipio_destinatario,
    
    -- Operação
    a.procnfe.nfe.infnfe.ide.natop AS natureza_operacao,
    a.procnfe.nfe.infnfe.ide.tpnf AS tipo_nf,
    CASE 
        WHEN a.procnfe.nfe.infnfe.ide.tpnf = 0 THEN 'Entrada'
        WHEN a.procnfe.nfe.infnfe.ide.tpnf = 1 THEN 'Saida'
        ELSE 'Indefinido'
    END AS entrada_saida,
    CASE 
        WHEN a.procnfe.nfe.infnfe.ide.indfinal = 1 THEN 'Consumidor final'
        ELSE 'Normal'
    END AS tipo_consumidor,
    
    -- Itens
    b._nitem AS numero_item,
    b.prod.cprod AS codigo_produto,
    b.prod.xprod AS descricao_produto,
    b.prod.ncm AS ncm,
    b.prod.cfop AS cfop,
    
    -- Valores
    CAST(COALESCE(b.prod.vprod, 0) AS DECIMAL(15,2)) AS valor_produto,
    CAST(COALESCE(b.prod.vfrete, 0) AS DECIMAL(15,2)) AS valor_frete,
    CAST(COALESCE(b.prod.vseg, 0) AS DECIMAL(15,2)) AS valor_seguro,
    CAST(COALESCE(b.prod.vdesc, 0) AS DECIMAL(15,2)) AS valor_desconto,
    CAST(COALESCE(b.prod.voutro, 0) AS DECIMAL(15,2)) AS valor_outras_despesas,
    
    -- ICMS
    b.imposto.icms.resumo.cst AS cst_icms,
    CAST(COALESCE(b.imposto.icms.resumo.vbc, 0) AS DECIMAL(15,2)) AS bc_icms,
    CAST(COALESCE(b.imposto.icms.resumo.picms, 0) AS DECIMAL(7,4)) AS aliquota_icms,
    CAST(COALESCE(b.imposto.icms.resumo.vicms, 0) AS DECIMAL(15,2)) AS valor_icms,
    CAST(COALESCE(b.imposto.icms.resumo.vcredicmssn, 0) AS DECIMAL(15,2)) AS valor_credito_sn,
    
    -- Totais NFe
    CAST(COALESCE(a.procnfe.nfe.infnfe.total.icmstot.vnf, 0) AS DECIMAL(15,2)) AS total_nfe,
    
    -- Flags cadastro
    IF(cnpj_emit IN ('{cnpj_in_clause}'), 1, 0) AS emitente_no_cadastro,
    IF(cnpj_dest IN ('{cnpj_in_clause}'), 1, 0) AS destinatario_no_cadastro

FROM (
    SELECT 
        *,
        REGEXP_REPLACE(TRIM(CAST(procnfe.nfe.infnfe.emit.cnpj AS STRING)), '[^0-9]', '') AS cnpj_emit,
        REGEXP_REPLACE(TRIM(CAST(procnfe.nfe.infnfe.dest.cnpj AS STRING)), '[^0-9]', '') AS cnpj_dest
    FROM nfe.nfe
    WHERE situacao = 1
      AND (ano_emissao * 100 + mes_emissao) >= {PERIODO_INICIO_AJUSTADO}
      AND (ano_emissao * 100 + mes_emissao) <= {PERIODO_FIM}
) a
LATERAL VIEW EXPLODE(a.procnfe.nfe.infnfe.det) exploded_table AS b
WHERE cnpj_emit IN ('{cnpj_in_clause}')
   OR cnpj_dest IN ('{cnpj_in_clause}')
"""

try:
    spark.sql(query_nfe_view)
    print("✓ View criada")
    
    # Verificar SEM executar count completo (muito pesado)
    print("\n✓ View NFe criada com sucesso")
    print(f"Período: {PERIODO_INICIO_AJUSTADO} até {PERIODO_FIM}")
    
    # Amostra para verificar
    print("\nAmostra dos dados (5 registros):")
    spark.sql("SELECT periodo_ref, cnpj_emitente, cnpj_destinatario, entrada_saida, valor_produto FROM vw_nfe_processada LIMIT 5").show(truncate=False)
    
except Exception as e:
    print(f"✗ Erro: {e}")
    raise

print("\n✓ Pronto para próximas etapas!")


CRIANDO VIEW DE NFe PROCESSADA (OTIMIZADA)
Fazendo REFRESH da tabela nfe.nfe...
✓ REFRESH executado
CNPJs no cadastro: 2

⚠️ ATENÇÃO: Pulando período 2020 devido a arquivos corrompidos
Período ajustado: 202101 até 202509
✓ View criada

✓ View NFe criada com sucesso
Período: 202101 até 202509

Amostra dos dados (5 registros):


[Stage 16:======================================================>(99 + 1) / 100]

+-----------+--------------+-----------------+-------------+-------------+
|periodo_ref|cnpj_emitente |cnpj_destinatario|entrada_saida|valor_produto|
+-----------+--------------+-----------------+-------------+-------------+
|202101     |32372396000194|NULL             |Saida        |7670.00      |
|202101     |18563423000185|32372396000194   |Saida        |37498.65     |
|202101     |32372396000194|NULL             |Saida        |1770.00      |
|202101     |32372396000194|NULL             |Saida        |10192.00     |
|202101     |32372396000194|NULL             |Saida        |2565.00      |
+-----------+--------------+-----------------+-------------+-------------+


✓ Pronto para próximas etapas!


In [ ]:
# ===================================================================
# CÉLULA 6: INTEGRAR CFOP (Execute após a célula 5 terminar)
# ===================================================================
print("\n" + "=" * 80)
print("INTEGRANDO INFORMAÇÕES DE CFOP")
print("=" * 80)

query_cfop_view = """
CREATE OR REPLACE TEMPORARY VIEW vw_nfe_com_cfop AS
SELECT 
    nfe.*,
    cfop.conta,
    cfop.descricaocfop,
    cfop.indcom,
    cfop.movimento
FROM vw_nfe_processada nfe
LEFT JOIN niat.tabela_cfop cfop 
    ON nfe.cfop = cfop.cfop
"""

spark.sql(query_cfop_view)
print("✓ View vw_nfe_com_cfop criada")

# Verificar
total = spark.sql("SELECT COUNT(*) as total FROM vw_nfe_com_cfop").collect()[0]['total']
print(f"✓ Total: {total:,} registros")

# Top CFOPs
print("\nTop 10 CFOPs:")
spark.sql("""
    SELECT 
        cfop,
        descricaocfop,
        conta,
        entrada_saida,
        COUNT(*) as qtde,
        ROUND(SUM(valor_produto), 2) as vl_total
    FROM vw_nfe_com_cfop
    GROUP BY cfop, descricaocfop, conta, entrada_saida
    ORDER BY qtde DESC
    LIMIT 10
""").show(truncate=False)


INTEGRANDO INFORMAÇÕES DE CFOP
✓ View vw_nfe_com_cfop criada


✓ Total: 3,818 registros

Top 10 CFOPs:


[Stage 26:==>       (6201 + 28) / 27177][Stage 27:>                 (0 + 0) / 1]

In [ ]:
# ===================================================================
# CÉLULA 6: INTEGRAR CFOP (CORRIGIDA E Refatorada - API)
# ===================================================================
print("\n" + "=" * 80)
print("INTEGRANDO INFORMAÇÕES DE CFOP")
print("=" * 80)

# Primeiro, carregar a tabela CFOP para memória
try:
    print("Carregando tabela CFOP...")
    df_cfop = spark.table("niat.tabela_cfop").select(
        "cfop", "conta", "descricaocfop", "indcom", 
        "movimento", "especial", "local", "mercenergtel"
    )
    
    # Criar view temporária e cachear
    df_cfop.createOrReplaceTempView("temp_cfop")
    spark.sql("CACHE TABLE temp_cfop")
    total_cfops = spark.table("temp_cfop").count()
    print(f"✓ Tabela CFOP cacheada: {total_cfops} registros")
    
except Exception as e:
    print(f"✗ Erro ao carregar tabela CFOP: {e}")
    print("\nCriando tabela CFOP básica manualmente...")
    
    # Se não conseguir acessar, criar uma básica
    cfop_data = [
        (1101, 0, "Compra para industrialização", "Industrialização", "Operação"),
        (1102, 0, "Compra para comercialização", "Comercialização", "Operação"),
        (5101, 1, "Venda de produção do estabelecimento", "Industrialização", "Operação"),
        (5102, 1, "Venda de mercadoria adquirida", "Comercialização", "Operação"),
        (5403, 1, "Venda de mercadoria em operação com não contribuinte", "Comercialização", "Operação"),
        (6101, 1, "Venda de produção interestadual", "Industrialização", "Operação"),
        (6102, 1, "Venda de mercadoria interestadual", "Comercialização", "Operação"),
    ]
    
    from pyspark.sql.types import StructType, StructField, IntegerType, StringType
    schema = StructType([
        StructField("cfop", IntegerType(), True),
        StructField("conta", IntegerType(), True),
        StructField("descricaocfop", StringType(), True),
        StructField("indcom", StringType(), True),
        StructField("movimento", StringType(), True),
    ])
    
    df_cfop = spark.createDataFrame(cfop_data, schema)
    df_cfop.createOrReplaceTempView("temp_cfop")
    spark.sql("CACHE TABLE temp_cfop")
    print("✓ Tabela CFOP básica criada e cacheada")

# Agora fazer o join com a view temporária usando API
df_nfe_processada = spark.table("vw_nfe_processada")
df_cfop_cached = spark.table("temp_cfop")

df_nfe_com_cfop = df_nfe_processada.join(
    df_cfop_cached,
    df_nfe_processada.cfop == df_cfop_cached.cfop,
    "left"
).select(
    df_nfe_processada["*"],
    df_cfop_cached["conta"],
    df_cfop_cached["descricaocfop"],
    df_cfop_cached["indcom"],
    df_cfop_cached["movimento"]
)

df_nfe_com_cfop.createOrReplaceTempView("vw_nfe_com_cfop")
print("✓ View vw_nfe_com_cfop criada")

# ⚠ REMOVIDO: O COUNT(*) aqui era lento e desnecessário.
# O Spark executará a view quando for realmente usada.

# Top CFOPs (para verificação)
print("\nTop 10 CFOPs (amostra):")
spark.sql("""
    SELECT 
        cfop,
        descricaocfop,
        conta,
        entrada_saida,
        COUNT(*) as qtde,
        ROUND(SUM(valor_produto), 2) as vl_total
    FROM vw_nfe_com_cfop
    GROUP BY cfop, descricaocfop, conta, entrada_saida
    ORDER BY qtde DESC
    LIMIT 10
""").show(truncate=False)

In [ ]:
# ===================================================================
# CÉLULA 7: CALCULAR ICMS
# ===================================================================
print("\n" + "=" * 80)
print("CALCULANDO ICMS - DÉBITOS E CRÉDITOS")
print("=" * 80)

query_icms = """
CREATE OR REPLACE TEMPORARY VIEW vw_nfe_com_icms AS
SELECT 
    *,
    -- BC total do item
    (valor_produto + valor_frete + valor_seguro + valor_outras_despesas - valor_desconto) AS bc_total_item,
    
    -- DÉBITO (Saída)
    CASE 
        WHEN entrada_saida = 'Saida' AND conta = 1 THEN
            CASE
                WHEN valor_icms > 0 THEN valor_icms
                WHEN bc_icms > 0 AND aliquota_icms > 0 THEN bc_icms * (aliquota_icms / 100)
                WHEN tipo_consumidor = 'Consumidor final' THEN 
                    (valor_produto + valor_frete + valor_seguro + valor_outras_despesas - valor_desconto) * 0.17
                WHEN uf_destinatario IN ('MG', 'PR', 'RJ', 'RS', 'SP') AND uf_destinatario != uf_emitente THEN
                    (valor_produto + valor_frete + valor_seguro + valor_outras_despesas - valor_desconto) * 0.12
                WHEN uf_destinatario != uf_emitente THEN
                    (valor_produto + valor_frete + valor_seguro + valor_outras_despesas - valor_desconto) * 0.07
                ELSE 
                    (valor_produto + valor_frete + valor_seguro + valor_outras_despesas - valor_desconto) * 0.12
            END
        ELSE 0
    END AS vl_icms_debito,
    
    CASE 
        WHEN entrada_saida = 'Saida' AND conta = 1 THEN
            COALESCE(bc_icms, valor_produto + valor_frete + valor_seguro + valor_outras_despesas - valor_desconto)
        ELSE 0
    END AS bc_icms_debito,
    
    -- CRÉDITO (Entrada)
    CASE 
        WHEN entrada_saida = 'Entrada' AND conta = -1 
             AND cst_icms NOT IN ('10', '15', '30', '40', '41', '53', '60', '61', '70', '102', '202', '203', '300', '500') THEN
            CASE
                WHEN valor_icms > 0 THEN valor_icms
                WHEN valor_credito_sn > 0 THEN valor_credito_sn
                ELSE 0
            END
        ELSE 0
    END AS vl_icms_credito,
    
    CASE 
        WHEN entrada_saida = 'Entrada' AND conta = -1 THEN
            COALESCE(bc_icms, valor_produto + valor_frete + valor_seguro + valor_outras_despesas - valor_desconto)
        ELSE 0
    END AS bc_icms_credito

FROM vw_nfe_com_cfop
"""

spark.sql(query_icms)
print("✓ View vw_nfe_com_icms criada")

# Resumo
print("\n📊 RESUMO ICMS:")
spark.sql("""
    SELECT 
        entrada_saida,
        COUNT(*) as qtde_itens,
        ROUND(SUM(bc_icms_debito), 2) as bc_debito,
        ROUND(SUM(vl_icms_debito), 2) as vl_debito,
        ROUND(SUM(bc_icms_credito), 2) as bc_credito,
        ROUND(SUM(vl_icms_credito), 2) as vl_credito
    FROM vw_nfe_com_icms
    GROUP BY entrada_saida
""").show(truncate=False)

In [ ]:
# ===================================================================
# CÉLULA 8: ANÁLISE POR ENTRADA/SAÍDA E EMITENTE/DESTINATÁRIO
# ===================================================================
print("\n" + "=" * 80)
print("SEPARANDO: ENTRADA vs SAÍDA | EMITENTE vs DESTINATÁRIO")
print("=" * 80)

# ENTRADA - DESTINATÁRIO (empresa do cadastro recebeu mercadoria)
df_entrada_dest = spark.sql("""
    SELECT *
    FROM vw_nfe_com_icms
    WHERE entrada_saida = 'Entrada'
      AND destinatario_no_cadastro = 1
""")
df_entrada_dest.createOrReplaceTempView("vw_entrada_destinatario")
# ⚠ REMOVIDO: count() desnecessário
print(f"✓ View 'vw_entrada_destinatario' criada.")


# ENTRADA - EMITENTE (empresa do cadastro emitiu nota de devolução/retorno)
df_entrada_emit = spark.sql("""
    SELECT *
    FROM vw_nfe_com_icms
    WHERE entrada_saida = 'Entrada'
      AND emitente_no_cadastro = 1
""")
df_entrada_emit.createOrReplaceTempView("vw_entrada_emitente")
# ⚠ REMOVIDO: count() desnecessário
print(f"✓ View 'vw_entrada_emitente' criada.")

# SAÍDA - EMITENTE (empresa do cadastro vendeu)
df_saida_emit = spark.sql("""
    SELECT *
    FROM vw_nfe_com_icms
    WHERE entrada_saida = 'Saida'
      AND emitente_no_cadastro = 1
""")
df_saida_emit.createOrReplaceTempView("vw_saida_emitente")
# ⚠ REMOVIDO: count() desnecessário
print(f"✓ View 'vw_saida_emitente' criada.")

# SAÍDA - DESTINATÁRIO (empresa do cadastro recebeu venda de terceiro)
df_saida_dest = spark.sql("""
    SELECT *
    FROM vw_nfe_com_icms
    WHERE entrada_saida = 'Saida'
      AND destinatario_no_cadastro = 1
""")
df_saida_dest.createOrReplaceTempView("vw_saida_destinatario")
# ⚠ REMOVIDO: count() desnecessário
print(f"✓ View 'vw_saida_destinatario' criada.")

print(f"\n✓ Views de separação prontas.")

In [ ]:
# ===================================================================
# CÉLULA 9: PROCESSAR CTe (CORRIGIDA - ESTRUTURA ICMS)
# ===================================================================
print("\n" + "=" * 80)
print("PROCESSANDO CTe - CRÉDITO DE FRETE")
print("=" * 80)

# Obter a lista de CNPJs (já deve estar em cache de 'cadastro_cnpj')
cnpj_list_cte = [row.cnpj for row in spark.sql("SELECT cnpj FROM cadastro_cnpj").collect()]
cnpj_in_clause = "','".join(cnpj_list_cte) # Para o SQL
print(f"Processando CTe para {len(cnpj_list_cte)} CNPJs.")

query_cte = f"""
CREATE OR REPLACE TEMPORARY VIEW vw_cte_processada AS
SELECT 
    -- Identificação
    a.chave AS chave_cte,
    a.ano_emissao AS ano_emissao,
    
    -- Período
    CONCAT(
        LPAD(MONTH(a.proccte.cte.infcte.ide.dhemi), 2, '0'), 
        '/', 
        YEAR(a.proccte.cte.infcte.ide.dhemi)
    ) AS periodo_ref,
    
    (YEAR(a.proccte.cte.infcte.ide.dhemi) * 100 + MONTH(a.proccte.cte.infcte.ide.dhemi)) AS periodo_ref_num,
    
    a.proccte.cte.infcte.ide.dhemi AS data_hora_emissao,
    
    -- Tomador (quem paga o frete)
    REGEXP_REPLACE(
        TRIM(CAST(a.proccte.cte.infcte.ide.toma4.cnpj AS STRING)), 
        '[^0-9]', ''
    ) AS cnpj_tomador,
    
    a.proccte.cte.infcte.ide.toma4.xnome AS nome_tomador,
    a.proccte.cte.infcte.ide.toma4.toma AS indicador_tomador,
    
    -- Remetente
    REGEXP_REPLACE(
        TRIM(CAST(a.proccte.cte.infcte.rem.cnpj AS STRING)), 
        '[^0-9]', ''
    ) AS cnpj_remetente,
    
    a.proccte.cte.infcte.rem.xnome AS nome_remetente,
    
    -- Destinatário
    REGEXP_REPLACE(
        TRIM(CAST(a.proccte.cte.infcte.dest.cnpj AS STRING)), 
        '[^0-9]', ''
    ) AS cnpj_destinatario,
    
    a.proccte.cte.infcte.dest.xnome AS nome_destinatario,
    
    -- Emitente (transportadora)
    REGEXP_REPLACE(
        TRIM(CAST(a.proccte.cte.infcte.emit.cnpj AS STRING)), 
        '[^0-9]', ''
    ) AS cnpj_emitente,
    
    a.proccte.cte.infcte.emit.xnome AS nome_emitente,
    a.proccte.cte.infcte.emit.ie AS ie_emitente,
    
    -- CFOP e Natureza
    a.proccte.cte.infcte.ide.cfop AS cfop,
    a.proccte.cte.infcte.ide.natop AS natureza_operacao,
    a.proccte.cte.infcte.ide.modal AS modal,
    a.proccte.cte.infcte.ide.tpserv AS tipo_servico,
    
    -- Valores
    CAST(COALESCE(a.proccte.cte.infcte.vprest.vtprest, 0) AS DECIMAL(15,2)) AS valor_total_servico,
    
    -- ICMS - Estrutura CORRETA baseada no describe
    CAST(COALESCE(
        a.proccte.cte.infcte.imp.icms.icms00.vicms,
        a.proccte.cte.infcte.imp.icms.icms20.vicms,
        a.proccte.cte.infcte.imp.icms.icms60.vcred,
        a.proccte.cte.infcte.imp.icms.icms90.vcred,
        a.proccte.cte.infcte.imp.icms.cst00.vicms,
        a.proccte.cte.infcte.imp.icms.cst20.vicms,
        a.proccte.cte.infcte.imp.icms.cst80.vcred,
        a.proccte.cte.infcte.imp.icms.cst90.vcred,
        0
    ) AS DECIMAL(15,2)) AS valor_icms,
    
    CAST(COALESCE(
        a.proccte.cte.infcte.imp.icms.icms00.vbc,
        a.proccte.cte.infcte.imp.icms.icms20.vbc,
        a.proccte.cte.infcte.imp.icms.icms90.vbc,
        a.proccte.cte.infcte.imp.icms.cst00.vbc,
        a.proccte.cte.infcte.imp.icms.cst20.vbc,
        a.proccte.cte.infcte.imp.icms.cst80.vbc,
        a.proccte.cte.infcte.imp.icms.cst90.vbc,
        0
    ) AS DECIMAL(15,2)) AS bc_icms,
    
    -- CST
    COALESCE(
        a.proccte.cte.infcte.imp.icms.icms00.cst,
        a.proccte.cte.infcte.imp.icms.icms20.cst,
        a.proccte.cte.infcte.imp.icms.icms45.cst,
        a.proccte.cte.infcte.imp.icms.icms60.cst,
        a.proccte.cte.infcte.imp.icms.icms90.cst,
        a.proccte.cte.infcte.imp.icms.icmssn.cst,
        a.proccte.cte.infcte.imp.icms.cst00.cst,
        a.proccte.cte.infcte.imp.icms.cst20.cst,
        a.proccte.cte.infcte.imp.icms.cst45.cst,
        a.proccte.cte.infcte.imp.icms.cst80.cst,
        a.proccte.cte.infcte.imp.icms.cst90.cst
    ) AS cst_icms

FROM cte.cte a
WHERE a.situacao = 1
  AND a.ano_emissao >= {PERIODO_INICIO // 100}
  AND a.ano_emissao <= {PERIODO_FIM // 100}
  AND a.proccte.cte.infcte.ide.toma4.cnpj IS NOT NULL
"""

try:
    spark.sql(query_cte)
    print("✓ View vw_cte_processada criada")
    
    # Filtrar CTe relevantes
    query_cte_filtrado = f"""
    CREATE OR REPLACE TEMPORARY VIEW vw_cte_filtrado AS
    SELECT 
        *,
        IF(cnpj_tomador IN ('{cnpj_in_clause}'), 1, 0) AS tomador_no_cadastro,
        IF(cnpj_destinatario IN ('{cnpj_in_clause}'), 1, 0) AS destinatario_no_cadastro,
        IF(cnpj_remetente IN ('{cnpj_in_clause}'), 1, 0) AS remetente_no_cadastro
    FROM vw_cte_processada
    WHERE periodo_ref_num >= {PERIODO_INICIO}
      AND periodo_ref_num <= {PERIODO_FIM}
      AND (
          cnpj_tomador IN ('{cnpj_in_clause}')
          OR cnpj_destinatario IN ('{cnpj_in_clause}')
          OR cnpj_remetente IN ('{cnpj_in_clause}')
      )
    """
    
    spark.sql(query_cte_filtrado)
    print(f"✓ View 'vw_cte_filtrado' criada.")
    
    # ⚠ REMOVIDO: count() desnecessário.
    # Vamos verificar com um LIMIT 5 se há dados.
    
    print("\nVerificando amostra de CTe (LIMIT 5):")
    amostra_cte = spark.sql("SELECT * FROM vw_cte_filtrado LIMIT 5").collect()
    
    if len(amostra_cte) > 0:
        print(f"✓ CTe encontrados. Exibindo {len(amostra_cte)} registro(s) de amostra:")
        spark.sql("SELECT * FROM vw_cte_filtrado LIMIT 5").show(truncate=False)

        # Resumo por período
        print("\nResumo CTe por período:")
        spark.sql("""
            SELECT 
                periodo_ref,
                COUNT(*) as qtde_cte,
                ROUND(SUM(valor_total_servico), 2) as vl_total_servico,
                ROUND(SUM(valor_icms), 2) as vl_icms_credito
            FROM vw_cte_filtrado
            GROUP BY periodo_ref
            ORDER BY periodo_ref
        """).show(20, truncate=False)
        
    else:
        print("\n⚠ Nenhum CTe encontrado para os CNPJs do cadastro no período")

except Exception as e:
    print(f"✗ Erro ao processar CTe: {e}")
    print("\n⚠ Criando view CTe vazia para continuar processamento...")
    
    # Criar view vazia
    spark.sql("""
        CREATE OR REPLACE TEMPORARY VIEW vw_cte_filtrado AS
        SELECT 
            CAST(NULL AS STRING) AS periodo_ref,
            CAST(NULL AS INT) AS periodo_ref_num,
            CAST(NULL AS STRING) AS cnpj_tomador,
            CAST(0 AS DECIMAL(15,2)) AS valor_icms,
            CAST(0 AS INT) AS tomador_no_cadastro
        WHERE 1=0
    """)
    
    print("✓ View CTe vazia criada - processamento continuará sem dados de CTe")

In [ ]:
# ===================================================================
# CÉLULA 10: CARREGAR PGDAS-D (Refatorado - API)
# ===================================================================
print("\n" + "=" * 80)
print("CARREGANDO DADOS PGDAS-D (SIMPLES NACIONAL)")
print("=" * 80)

# Lista de CNPJs já deve estar em cache de 'cadastro_cnpj'
cnpj_list_pgdas = [row.cnpj for row in spark.sql("SELECT cnpj FROM cadastro_cnpj").collect()]
print(f"Processando PGDAS-D para {len(cnpj_list_pgdas)} CNPJs.")

try:
    df_pgdas_raw = spark.table("usr_sat_ods.sna_pgdasd_estabelecimento_raw")

    df_pgdas_filtrado = (df_pgdas_raw
        .withColumn("cnpj", regexp_replace(trim(col("nu_cnpj")), '[^0-9]', ''))
        .filter(
            (col("nu_per_ref") >= PERIODO_INICIO) &
            (col("nu_per_ref") <= PERIODO_FIM) &
            (col("cnpj").isin(cnpj_list_pgdas))
        )
        .select(
            col("cnpj"),
            col("nu_per_ref").alias("periodo_ref"),
            concat(
                lpad((col("nu_per_ref") % 100).cast("string"), 2, '0'),
                lit('/'),
                floor(col("nu_per_ref") / 100).cast("string")
            ).alias("periodo_formatado"),
            
            coalesce(col("vl_rec_bruta_estab"), lit(0)).cast("decimal(15,2)").alias("receita_bruta"),
            coalesce(col("vl_icms_sc"), lit(0)).cast("decimal(15,2)").alias("icms_declarado_sc")
        )
    )

    df_pgdas_filtrado.createOrReplaceTempView("vw_pgdas_filtrado")
    
    # ⚠ REMOVIDO: count() desnecessário.
    print(f"✓ View 'vw_pgdas_filtrado' criada.")

    # Mostrar amostra
    print("\nAmostra PGDAS-D:")
    spark.sql("""
        SELECT *
        FROM vw_pgdas_filtrado
        ORDER BY periodo_ref DESC
        LIMIT 10
    """).show(10, truncate=False)

except Exception as e:
    print(f"✗ Erro ao carregar PGDAS-D: {e}")
    print("\n⚠ Criando view PGDAS-D vazia para continuar processamento...")
    
    # Criar view vazia
    spark.sql("""
        CREATE OR REPLACE TEMPORARY VIEW vw_pgdas_filtrado AS
        SELECT 
            CAST(NULL AS STRING) AS cnpj,
            CAST(NULL AS INT) AS periodo_ref,
            CAST(NULL AS STRING) AS periodo_formatado,
            CAST(0 AS DECIMAL(15,2)) AS receita_bruta,
            CAST(0 AS DECIMAL(15,2)) AS icms_declarado_sc
        WHERE 1=0
    """)
    print("✓ View PGDAS-D vazia criada")

In [ ]:
# ===================================================================
# CÉLULA 11: CONSOLIDAR NOTIFICAÇÃO (✅ PADRÃO OURO)
# ===================================================================
print("\n" + "=" * 80)
print("GERANDO NOTIFICAÇÃO - APURAÇÃO MENSAL DE ICMS")
print("=" * 80)

query_notificacao = """
CREATE OR REPLACE TEMPORARY VIEW vw_notificacao AS
WITH apuracao_saida AS (
    SELECT 
        periodo_ref,
        cnpj_emitente AS cnpj,
        SUM(bc_icms_debito) AS bc_icms_saida,
        SUM(vl_icms_debito) AS vl_icms_saida
    FROM vw_saida_emitente
    GROUP BY periodo_ref, cnpj_emitente
),
apuracao_entrada AS (
    SELECT 
        periodo_ref,
        cnpj_destinatario AS cnpj,
        SUM(bc_icms_credito) AS bc_icms_entrada,
        SUM(vl_icms_credito) AS vl_icms_entrada
    FROM vw_entrada_destinatario
    GROUP BY periodo_ref, cnpj_destinatario
),
apuracao_cte AS (
    SELECT 
        periodo_ref_num AS periodo_ref,
        cnpj_tomador AS cnpj,
        SUM(valor_icms) AS vl_icms_cte
    FROM vw_cte_filtrado
    WHERE tomador_no_cadastro = 1
    GROUP BY periodo_ref_num, cnpj_tomador
)
SELECT 
    COALESCE(s.periodo_ref, e.periodo_ref, c.periodo_ref, p.periodo_ref) AS periodo,
    COALESCE(s.cnpj, e.cnpj, c.cnpj, p.cnpj) AS cnpj,
    CONCAT(
        LPAD(CAST(COALESCE(s.periodo_ref, e.periodo_ref, c.periodo_ref, p.periodo_ref) % 100 AS STRING), 2, '0'),
        '/',
        CAST(COALESCE(s.periodo_ref, e.periodo_ref, c.periodo_ref, p.periodo_ref) / 100 AS STRING)
    ) AS periodo_formatado,
    
    -- Débitos
    ROUND(COALESCE(s.bc_icms_saida, 0), 2) AS bc_icms_saida,
    ROUND(COALESCE(s.vl_icms_saida, 0), 2) AS vl_icms_saida,
    
    -- Créditos
    ROUND(COALESCE(e.bc_icms_entrada, 0), 2) AS bc_icms_entrada,
    ROUND(COALESCE(e.vl_icms_entrada, 0), 2) AS vl_icms_entrada,
    ROUND(COALESCE(c.vl_icms_cte, 0), 2) AS vl_icms_cte,
    
    -- PGDAS-D
    ROUND(COALESCE(p.receita_bruta, 0), 2) AS receita_bruta_declarada,
    ROUND(COALESCE(p.icms_declarado_sc, 0), 2) AS icms_declarado,
    
    -- Cálculos
    ROUND(COALESCE(s.bc_icms_saida, 0) - COALESCE(p.receita_bruta, 0), 2) AS receita_nao_declarada,
    ROUND(CASE 
              WHEN (COALESCE(s.bc_icms_saida, 0) - COALESCE(p.receita_bruta, 0)) > 0 
              THEN (COALESCE(s.bc_icms_saida, 0) - COALESCE(p.receita_bruta, 0)) * 0.17 
              ELSE 0 
          END, 2) AS vl_icms_receita_omitida,
    
    -- ICMS DEVIDO
    ROUND(
        COALESCE(s.vl_icms_saida, 0) +
        (CASE 
            WHEN (COALESCE(s.bc_icms_saida, 0) - COALESCE(p.receita_bruta, 0)) > 0 
            THEN (COALESCE(s.bc_icms_saida, 0) - COALESCE(p.receita_bruta, 0)) * 0.17 
            ELSE 0 
         END) -
        COALESCE(e.vl_icms_entrada, 0) -
        COALESCE(c.vl_icms_cte, 0) -
        COALESCE(p.icms_declarado_sc, 0),
        2
    ) AS vl_icms_devido,
    
    -- Vencimento (dia 10 do mês seguinte)
    DATE_FORMAT(
        ADD_MONTHS(
            CONCAT(
                CAST(COALESCE(s.periodo_ref, e.periodo_ref, c.periodo_ref, p.periodo_ref) / 100 AS INT), '-',
                LPAD(CAST(COALESCE(s.periodo_ref, e.periodo_ref, c.periodo_ref, p.periodo_ref) % 100 AS INT), 2, '0'), '-01'
            ),
            1
        ),
        'yyyy-MM-10'
    ) AS data_vencimento

FROM apuracao_saida s
FULL OUTER JOIN apuracao_entrada e 
    ON s.periodo_ref = e.periodo_ref AND s.cnpj = e.cnpj
FULL OUTER JOIN apuracao_cte c 
    ON COALESCE(s.periodo_ref, e.periodo_ref) = c.periodo_ref 
    AND COALESCE(s.cnpj, e.cnpj) = c.cnpj
FULL OUTER JOIN vw_pgdas_filtrado p 
    ON COALESCE(s.periodo_ref, e.periodo_ref, c.periodo_ref) = p.periodo_ref
    AND COALESCE(s.cnpj, e.cnpj, c.cnpj) = p.cnpj
ORDER BY cnpj, periodo
"""

# ===================================================================
# ✅ APLICAÇÃO DO PADRÃO OURO (Contar -> Decidir -> Limitar -> toPandas)
# ===================================================================
try:
    spark.sql(query_notificacao)
    print("✓ View vw_notificacao criada")
    
    # Materializar resultado para evitar reprocessamento
    print("\nMaterializando dados da notificação...")
    df_notificacao = spark.sql("SELECT * FROM vw_notificacao ORDER BY cnpj, periodo")
    df_notificacao.cache()  # Cachear para contar e agregar
    
    # 1. CONTAR
    total_periodos = df_notificacao.count()
    print(f"✓ Total de períodos apurados: {total_periodos}")
    
    LIMITE_PANDAS_GRAFICOS = 100000 # Limite seguro para gráficos
    df_notif_pandas = pd.DataFrame() # Inicializa vazio
    
    # 2. DECIDIR
    if total_periodos == 0:
        print("\n⚠ Nenhum período com dados para apuração")
        df_notif_pandas = pd.DataFrame()
        
    elif total_periodos > LIMITE_PANDAS_GRAFICOS:
        print(f"\n⚠ Muitos registros ({total_periodos:,}). Limitando a {LIMITE_PANDAS_GRAFICOS} para visualização...")
        # 3. LIMITAR E COLETAR
        df_notif_pandas = df_notificacao.limit(LIMITE_PANDAS_GRAFICOS).toPandas()
        
    else:
        print(f"\n✓ Coletando {total_periodos:,} registros para visualização...")
        # 3. COLETAR
        df_notif_pandas = df_notificacao.toPandas()
        
    # ===================================================================
    # ✅ CÁLCULO DE ESTATÍSTICAS SEGURO (Feito no Spark)
    # ===================================================================
    if total_periodos > 0:
        print("\n" + "=" * 80)
        print("ESTATÍSTICAS CONSOLIDADAS (Calculado via Spark)")
        print("=" * 80)

        # Calcular agregados no Spark (muito mais seguro)
        resumo_total_spark = df_notificacao.agg(
            spark_sum("vl_icms_devido").alias("total_devido"),
            spark_sum("vl_icms_saida").alias("total_debitos"),
            spark_sum("vl_icms_entrada").alias("total_cred_entrada"),
            spark_sum("vl_icms_cte").alias("total_cred_cte"),
            spark_sum("icms_declarado").alias("total_declarado"),
            spark_sum("receita_nao_declarada").alias("total_rec_omitida"),
            spark_sum("vl_icms_receita_omitida").alias("total_icms_rec_omitida")
        ).collect()[0]

        print(f"Total ICMS Devido: R$ {resumo_total_spark['total_devido']:,.2f}")
        print(f"Total Débitos (Saída): R$ {resumo_total_spark['total_debitos']:,.2f}")
        print(f"Total Créditos (Entrada): R$ {resumo_total_spark['total_cred_entrada']:,.2f}")
        print(f"Total Créditos (CTe): R$ {resumo_total_spark['total_cred_cte']:,.2f}")
        print(f"Total ICMS Declarado: R$ {resumo_total_spark['total_declarado']:,.2f}")
        print(f"Receita Não Declarada: R$ {resumo_total_spark['total_rec_omitida']:,.2f}")
        print(f"ICMS sobre Receita Omitida: R$ {resumo_total_spark['total_icms_rec_omitida']:,.2f}")

        # Resumo por CNPJ (Spark)
        print("\n" + "=" * 80)
        print("RESUMO POR CNPJ (Calculado via Spark)")
        print("=" * 80)
        
        resumo_cnpj_spark = df_notificacao.groupBy("cnpj").agg(
            spark_count("periodo").alias("Qtde_Períodos"),
            spark_sum("vl_icms_saida").alias("Total_Débitos"),
            spark_sum("vl_icms_entrada").alias("Total_Créd_Entrada"),
            spark_sum("vl_icms_cte").alias("Total_Créd_CTe"),
            spark_sum("icms_declarado").alias("Total_Declarado"),
            spark_sum("vl_icms_devido").alias("Total_Devido"),
            spark_sum("receita_nao_declarada").alias("Rec_Omitida")
        ).orderBy(desc("Total_Devido"))
        
        resumo_cnpj_spark.show(truncate=False)
        
    else:
        # Se não houver dados, df_notif_pandas já é um DataFrame vazio
        pass
        
except Exception as e:
    print(f"\n✗ Erro ao gerar notificação: {e}")
    df_notif_pandas = pd.DataFrame() # Garante que seja um DF vazio em caso de erro

# Limpar o cache (se não for usar mais)
# df_notificacao.unpersist()
# Vamos manter cacheado para a exportação na Célula 13

In [ ]:
# ===================================================================
# CÉLULA 12: VISUALIZAÇÕES E ANÁLISES
# ===================================================================
print("\n" + "=" * 80)
print("VISUALIZAÇÕES E ANÁLISES")
print("=" * 80)

import matplotlib.pyplot as plt
import seaborn as sns

# ✅ Verificação Padrão Ouro: Só plote se o DataFrame não estiver vazio
if not df_notif_pandas.empty:
    print(f"Gerando gráficos com base em {len(df_notif_pandas)} registros (limitados).")
    
    # Preparar dados
    df_plot = df_notif_pandas.copy()
    df_plot['periodo'] = df_plot['periodo'].astype(str)

    # Criar figura com múltiplos gráficos
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('ANÁLISE FISCAL - ICMS (Amostra Limitada)', fontsize=16, fontweight='bold')

    # Gráfico 1: Evolução do ICMS Devido
    ax1 = axes[0, 0]
    for cnpj in df_plot['cnpj'].unique():
        data = df_plot[df_plot['cnpj'] == cnpj]
        ax1.plot(data['periodo'], data['vl_icms_devido'], marker='o', label=f'CNPJ {cnpj}')
    ax1.set_title('Evolução do ICMS Devido', fontweight='bold')
    ax1.set_xlabel('Período')
    ax1.set_ylabel('Valor (R$)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)

    # Gráfico 2: Débitos vs Créditos
    ax2 = axes[0, 1]
    # Usar os dados já agregados pela célula anterior seria mais rápido
    # Mas para fins de plotagem temporal, vamos re-agregar o df_plot
    periodos_plot = df_plot['periodo'].unique()
    if len(periodos_plot) > 24:
        periodos_plot = periodos_plot[-24:] # Limitar aos últimos 24 períodos
        
    data_resumo = df_plot[df_plot['periodo'].isin(periodos_plot)].groupby('periodo').agg({
        'vl_icms_saida': 'sum',
        'vl_icms_entrada': 'sum',
        'vl_icms_cte': 'sum'
    }).reset_index()

    x = range(len(data_resumo))
    width = 0.25
    ax2.bar([i - width for i in x], data_resumo['vl_icms_saida'], width, label='Débito (Saída)', color='red', alpha=0.7)
    ax2.bar(x, data_resumo['vl_icms_entrada'], width, label='Crédito (Entrada)', color='green', alpha=0.7)
    ax2.bar([i + width for i in x], data_resumo['vl_icms_cte'], width, label='Crédito (CTe)', color='blue', alpha=0.7)
    ax2.set_title(f'Débitos vs Créditos - (Últimos {len(periodos_plot)} Períodos)', fontweight='bold')
    ax2.set_xlabel('Período')
    ax2.set_ylabel('Valor (R$)')
    ax2.set_xticks(x)
    ax2.set_xticklabels(data_resumo['periodo'], rotation=45)
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # Gráfico 3: Receita Declarada vs Receita Apurada
    ax3 = axes[1, 0]
    data_receita = df_plot[df_plot['periodo'].isin(periodos_plot)].groupby('periodo').agg({
        'bc_icms_saida': 'sum',
        'receita_bruta_declarada': 'sum'
    }).reset_index()
    
    ax3.plot(data_receita['periodo'], data_receita['bc_icms_saida'], marker='o', label='BC ICMS (Apurada)', linewidth=2)
    ax3.plot(data_receita['periodo'], data_receita['receita_bruta_declarada'], marker='s', label='Receita Declarada', linewidth=2)
    ax3.fill_between(range(len(data_receita)), data_receita['bc_icms_saida'], data_receita['receita_bruta_declarada'], 
                      where=(data_receita['bc_icms_saida'] > data_receita['receita_bruta_declarada']), 
                      alpha=0.3, color='red', label='Diferença (Omissão)')
    ax3.set_title(f'Receita Declarada vs Apurada (Últimos {len(periodos_plot)} Períodos)', fontweight='bold')
    ax3.set_xlabel('Período')
    ax3.set_ylabel('Valor (R$)')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.tick_params(axis='x', rotation=45)

    # Gráfico 4: Composição do ICMS Devido
    ax4 = axes[1, 1]
    # Usa os totais do DataFrame (limitado)
    totais = df_plot.sum() 
    componentes = ['Débito\n(Saída)', 'Receita\nOmitida', 'Declarado', 'Crédito\n(Entrada)', 'Crédito\n(CTe)']
    valores = [
        totais['vl_icms_saida'],
        totais['vl_icms_receita_omitida'],
        -totais['icms_declarado'],
        -totais['vl_icms_entrada'],
        -totais['vl_icms_cte']
    ]
    cores = ['red', 'orange', 'blue', 'green', 'cyan']
    ax4.bar(componentes, valores, color=cores, alpha=0.7)
    ax4.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax4.set_title('Composição do ICMS Devido (Total da Amostra)', fontweight='bold')
    ax4.set_ylabel('Valor (R$)')
    ax4.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()
    plt.show()

    # ⚠ REMOVIDO: Bloco de estatísticas agora está na célula anterior e é calculado via Spark.
    
else:
    print("⚠ df_notif_pandas está vazio. Nenhum gráfico será gerado.")

In [ ]:
# ===================================================================
# CÉLULA 13: EXPORTAR RESULTADOS (✅ PADRÃO OURO)
# ===================================================================
print("\n" + "=" * 80)
print("EXPORTANDO RESULTADOS PARA REDE LOCAL")
print("=" * 80)

import getpass
import smbclient
import os
import logging
from datetime import datetime

# Configurar logging
logging.getLogger('smbprotocol').setLevel(logging.WARNING)

# =============================================================================
# CONFIGURAÇÃO DA CONEXÃO SMB
# =============================================================================
server = "sef.sc.gov.br"
user = "tsevero" # ✅ Ajuste se necessário

try:
    pwd = getpass.getpass(f"Digite a senha de rede para {user}@{server}: ")
    smbclient.register_session(server, username=user, password=pwd)
    print(f"✓ Sessão SMB registrada com sucesso para {user}!")
except Exception as e:
    print(f"✗ Falha ao registrar sessão SMB: {e}")
    raise

# =============================================================================
# DEFINIR DIRETÓRIO DE DESTINO
# =============================================================================
# Criar pasta com timestamp para organizar
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
dest_dir = r"\\sef.sc.gov.br\DFS\GERFE08\Backup Severo\JUPYTER\Analise_Fiscal_NFe_CTe"
dest_dir_timestamped = os.path.join(dest_dir, f"GEI_{timestamp}").replace(os.sep, '\\')

print(f"\nCriando pasta de destino: {dest_dir_timestamped}")
try:
    if not smbclient.path.isdir(dest_dir):
        smbclient.makedirs(dest_dir)
    smbclient.makedirs(dest_dir_timestamped)
    print("✓ Pasta criada com sucesso")
except Exception as e:
    print(f"✗ Erro ao criar pasta: {e}")
    raise

# =============================================================================
# ✅ FUNÇÕES DE EXPORTAÇÃO "PADRÃO OURO"
# =============================================================================

# Limite máximo de linhas para um arquivo Excel
LIMITE_EXPORT_EXCEL = 1048575 

def salvar_pandas_na_rede(df_pandas, nome_arquivo, caminho_destino):
    """
    Salva um DataFrame PANDAS (já coletado) em um arquivo Excel 
    passando por um temp local e copiando via SMB.
    """
    try:
        if len(df_pandas) == 0:
            print(f"  ⚠ DataFrame vazio, pulando: {nome_arquivo}")
            return False
            
        caminho_completo = os.path.join(caminho_destino, nome_arquivo).replace(os.sep, '\\')
        temp_file = f"/tmp/{nome_arquivo}" # Arquivo temporário local no nó driver
        
        # Salvar como Excel localmente
        print(f"  ... salvando {len(df_pandas):,} linhas no temp local...")
        with pd.ExcelWriter(temp_file, engine='openpyxl') as writer:
            df_pandas.to_excel(writer, index=False, sheet_name='Dados')
        
        # Copiar para a rede
        print(f"  ... copiando para {caminho_completo} ...")
        with open(temp_file, 'rb') as f_local:
            with smbclient.open_file(caminho_completo, mode='wb') as f_remoto:
                f_remoto.write(f_local.read())
        
        # Limpar arquivo temporário
        os.remove(temp_file)
        
        print(f"  ✓ {nome_arquivo}: {len(df_pandas):,} registros exportados")
        return True
        
    except Exception as e:
        print(f"  ✗ Erro ao salvar/copiar {nome_arquivo}: {e}")
        return False

def exportar_view_spark_segura(view_name, order_by_cols, nome_arquivo, caminho_destino):
    """
    Aplica a lógica de segurança (Contar -> Decidir -> Limitar -> toPandas)
    antes de chamar a função de exportação.
    """
    print(f"\nProcessando exportação: {nome_arquivo} (View: {view_name})")
    
    try:
        df_spark = spark.table(view_name)
        
        # 1. CONTAR
        print(f"  ... contando registros em '{view_name}'...")
        total = df_spark.count()
        
        # 2. DECIDIR
        if total == 0:
            print(f"  ⚠ View '{view_name}' está vazia. Pulando.")
            arquivos_erro.append(f"{nome_arquivo} (Vazio)")
            return False
        
        df_para_exportar = df_spark
        
        if total > LIMITE_EXPORT_EXCEL:
            print(f"  ⚠ Muitos registros ({total:,}). Exportando APENAS os primeiros {LIMITE_EXPORT_EXCEL:,}...")
            # É crucial ter um ORDER BY para o LIMIT ser determinístico
            if order_by_cols:
                # Usar Window Function para "LIMIT" determinístico antes de ordenar
                # Isto é mais robusto para performance do que OrderBy + Limit
                window_spec = Window.orderBy(*order_by_cols)
                df_para_exportar = (df_para_exportar
                                    .withColumn("row_num", row_number().over(window_spec))
                                    .where(col("row_num") <= LIMITE_EXPORT_EXCEL)
                                    .drop("row_num"))
            else:
                df_para_exportar = df_para_exportar.limit(LIMITE_EXPORT_EXCEL)
        else:
            print(f"  ✓ Coletando {total:,} registros para exportação.")
            if order_by_cols:
                df_para_exportar = df_para_exportar.orderBy(*order_by_cols)

        # 3. toPandas (Agora é seguro!)
        print("  ... coletando dados para o Pandas...")
        df_pandas = df_para_exportar.toPandas()
        
        # 4. Exportar
        if salvar_pandas_na_rede(df_pandas, nome_arquivo, caminho_destino):
            arquivos_sucesso.append(nome_arquivo)
            return True
        else:
            arquivos_erro.append(f"{nome_arquivo} (Erro ao Salvar)")
            return False

    except Exception as e:
        print(f"  ✗ Erro GERAL ao processar '{view_name}': {e}")
        arquivos_erro.append(f"{nome_arquivo} (Erro Spark: {e})")
        return False

# =============================================================================
# EXPORTAR ARQUIVOS PRINCIPAIS
# =============================================================================
print("\n" + "=" * 80)
print("INICIANDO EXPORTAÇÃO DOS ARQUIVOS")
print("=" * 80)

arquivos_sucesso = []
arquivos_erro = []

# Cachear as views que serão exportadas para acelerar os counts()
print("Cacheando views para exportação...")
views_para_exportar = [
    "vw_notificacao", "vw_entrada_destinatario", "vw_saida_emitente", 
    "vw_cte_filtrado", "vw_pgdas_filtrado", "vw_nfe_com_icms"
]
for v in views_para_exportar:
    try:
        spark.table(v).cache()
        print(f"✓ View '{v}' cacheada.")
    except Exception as e:
        print(f"⚠ Não foi possível cachear a view '{v}': {e}")

# 1. NOTIFICAÇÃO (Apuração Mensal)
exportar_view_spark_segura(
    "vw_notificacao", 
    ["cnpj", "periodo"], 
    "01_Notificacao.xlsx", 
    dest_dir_timestamped
)

# 2. ENTRADA DESTINATÁRIO (Créditos)
exportar_view_spark_segura(
    "vw_entrada_destinatario", 
    ["periodo_ref", "chave_nfe"], 
    "02_Entrada_Destinatario.xlsx", 
    dest_dir_timestamped
)

# 3. SAÍDA EMITENTE (Débitos)
exportar_view_spark_segura(
    "vw_saida_emitente", 
    ["periodo_ref", "chave_nfe"], 
    "03_Saida_Emitente.xlsx", 
    dest_dir_timestamped
)

# 4. CTe (Crédito de Frete)
exportar_view_spark_segura(
    "vw_cte_filtrado", 
    ["periodo_ref", "chave_cte"], 
    "04_CTe.xlsx", 
    dest_dir_timestamped
)

# 5. PGDAS-D (Simples Nacional)
exportar_view_spark_segura(
    "vw_pgdas_filtrado", 
    ["cnpj", "periodo_ref"], 
    "05_PGDAS_D.xlsx", 
    dest_dir_timestamped
)

# 6. RESUMO POR CNPJ (Já calculado em Spark na Célula 11)
print("\nProcessando exportação: 06_Resumo_por_CNPJ.xlsx")
try:
    # Re-executar o resumo da Célula 11 para garantir
    df_resumo_cnpj = spark.sql("""
        SELECT 
            cnpj,
            COUNT(DISTINCT periodo) as qtde_periodos,
            ROUND(SUM(vl_icms_saida), 2) as total_debitos,
            ROUND(SUM(vl_icms_entrada), 2) as total_creditos_entrada,
            ROUND(SUM(vl_icms_cte), 2) as total_creditos_cte,
            ROUND(SUM(icms_declarado), 2) as total_declarado,
            ROUND(SUM(vl_icms_devido), 2) as total_devido,
            ROUND(SUM(receita_nao_declarada), 2) as total_receita_omitida
        FROM vw_notificacao
        GROUP BY cnpj
        ORDER BY total_devido DESC
    """)
    df_pandas_resumo = df_resumo_cnpj.toPandas() # Seguro, pois é agregado
    
    if salvar_pandas_na_rede(df_pandas_resumo, "06_Resumo_por_CNPJ.xlsx", dest_dir_timestamped):
        arquivos_sucesso.append("06_Resumo_por_CNPJ.xlsx")
    else:
        arquivos_erro.append("06_Resumo_por_CNPJ.xlsx (Erro ao Salvar)")
except Exception as e:
    print(f"  ✗ Erro GERAL ao processar '06_Resumo_por_CNPJ.xlsx': {e}")
    arquivos_erro.append(f"06_Resumo_por_CNPJ.xlsx (Erro Spark: {e})")

# 7. TOP CFOPs
print("\nProcessando exportação: 07_Top_CFOPs.xlsx")
try:
    df_top_cfops = spark.sql("""
        SELECT 
            cfop, descricaocfop, entrada_saida, conta,
            COUNT(*) as qtde_itens,
            ROUND(SUM(valor_produto), 2) as valor_total
        FROM vw_nfe_com_icms
        GROUP BY cfop, descricaocfop, entrada_saida, conta
        ORDER BY qtde_itens DESC
        LIMIT 1000
    """)
    df_pandas_cfop = df_top_cfops.toPandas() # Seguro, pois é limitado
    
    if salvar_pandas_na_rede(df_pandas_cfop, "07_Top_CFOPs.xlsx", dest_dir_timestamped):
        arquivos_sucesso.append("07_Top_CFOPs.xlsx")
    else:
        arquivos_erro.append("07_Top_CFOPs.xlsx (Erro ao Salvar)")
except Exception as e:
    print(f"  ✗ Erro GERAL ao processar '07_Top_CFOPs.xlsx': {e}")
    arquivos_erro.append(f"07_Top_CFOPs.xlsx (Erro Spark: {e})")

# 8. TOP PRODUTOS
print("\nProcessando exportação: 08_Top_Produtos.xlsx")
try:
    df_top_produtos = spark.sql("""
        SELECT 
            descricao_produto, ncm,
            COUNT(*) as qtde_vendas,
            ROUND(SUM(valor_produto), 2) as valor_total
        FROM vw_nfe_com_icms
        WHERE entrada_saida = 'Saida' AND emitente_no_cadastro = 1
        GROUP BY descricao_produto, ncm
        ORDER BY valor_total DESC
        LIMIT 1000
    """)
    df_pandas_produtos = df_top_produtos.toPandas() # Seguro, pois é limitado
    
    if salvar_pandas_na_rede(df_pandas_produtos, "08_Top_Produtos.xlsx", dest_dir_timestamped):
        arquivos_sucesso.append("08_Top_Produtos.xlsx")
    else:
        arquivos_erro.append("08_Top_Produtos.xlsx (Erro ao Salvar)")
except Exception as e:
    print(f"  ✗ Erro GERAL ao processar '08_Top_Produtos.xlsx': {e}")
    arquivos_erro.append(f"08_Top_Produtos.xlsx (Erro Spark: {e})")
    
# =============================================================================
# RELATÓRIO FINAL
# =============================================================================
print("\n" + "=" * 80)
print("RELATÓRIO DE EXPORTAÇÃO")
print("=" * 80)
print(f"\nPasta de destino: {dest_dir_timestamped}")
print(f"\nArquivos exportados com sucesso: {len(arquivos_sucesso)}")
for arq in arquivos_sucesso:
    print(f"  ✓ {arq}")

if arquivos_erro:
    print(f"\nArquivos com erro ou vazios: {len(arquivos_erro)}")
    for arq in arquivos_erro:
        print(f"  ✗ {arq}")

print("\n" + "=" * 80)
print("EXPORTAÇÃO CONCLUÍDA!")
print("=" * 80)

# Limpar todo o cache do Spark
print("Limpando cache do Spark...")
spark.catalog.clearCache()
print("✓ Cache limpo.")